# Layerwise learning for Quantum Neural Nets

In [1]:
import random
import collections
import matplotlib.pyplot as plt

# Pennylane
import pennylane as qml
from pennylane import numpy as np

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
torch.cuda.is_available()

/home/felipeoyarce/Desktop/layerwise-learning/.venv/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

### Data pre-processing

In [3]:
data_transforms = transforms.Compose([transforms.Resize(3), #resize to a 2x2 image
                                      transforms.ToTensor(), #convert to tensor
                                      transforms.Normalize(
                                     (0.1307,), (0.3081,)),
                                      transforms.Lambda(lambda x: torch.flatten(x)) #obtain a one dimensional vector
                                     ])

In [4]:
batch_size = 128
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=data_transforms)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=data_transforms)

# Change labels of digits '3' and '6' to be 0 and 1, respectively.
train_set.targets[train_set.targets == 1] = 10
train_set.targets[train_set.targets == 0] = 10
train_set.targets[train_set.targets == 3] = 1
train_set.targets[train_set.targets == 6] = -1

test_set.targets[test_set.targets == 1] = 10
test_set.targets[test_set.targets == 0] = 10
test_set.targets[test_set.targets == 3] = 1
test_set.targets[test_set.targets == 6] = -1

# Filter to just images of '3's and '6's
subset_indices_train = ((train_set.targets == -1) + (train_set.targets == 1)).nonzero().view(-1)
subset_indices_test = ((test_set.targets == -1) + (test_set.targets == 1)).nonzero().view(-1)

print(len(subset_indices_test))

# Select just a subset of the training set
NUM_EXAMPLES = 128
subset_indices_train = subset_indices_train[:NUM_EXAMPLES]

# DataLoaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices_train))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False,
                                         sampler=SubsetRandomSampler(subset_indices_test))

1968


/home/felipeoyarce/Desktop/layerwise-learning/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)


In [5]:
for x, y in train_loader:
    print(x)
    print(y)

tensor([[-0.3351,  0.5177,  0.1995,  ...,  0.3013,  0.5686, -0.2842],
        [-0.3224,  0.1995, -0.2969,  ..., -0.3224,  0.3649, -0.1696],
        [-0.1824,  0.0976, -0.3860,  ..., -0.3351,  0.1740, -0.1951],
        ...,
        [-0.3478,  0.6068, -0.2206,  ..., -0.1187,  0.4922, -0.2842],
        [-0.0042,  0.2886, -0.3606,  ..., -0.2969,  0.3777,  0.0085],
        [-0.3224,  0.2758, -0.2842,  ..., -0.2587,  0.3904, -0.1187]])
tensor([ 1, -1, -1, -1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,
        -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1,  1,
        -1,  1,  1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1,  1,
        -1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1,  1, -1,  1,
         1, -1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,  1,  1, -1,
         1, -1,  1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1,  1,  1,
        -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,
      

In [6]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

### Variational circuit

We first define some quantum layers that will compose the quantum circuit.

In [7]:
n_qubits = 9
n_layer_steps = 2
n_layers_to_add = 2
batch_size = 128
epochs = 20

In [8]:
dev = qml.device("default.qubit", wires=n_qubits)

In [9]:
def total_elements(array_list):
    flattened = [val for sublist in array_list for val in sublist]
    return len(flattened)

## Phase I

In [10]:
def apply_layer(gates, weights):
    
    for i in range(n_qubits): 
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
#template for non-trainable part of the quantum circuit
def frozen_layers(frozen_layer_gates, frozen_layer_weights): 

    for i in range(len(frozen_layer_gates)):
        apply_layer(frozen_layer_gates[i], frozen_layer_weights[i])

@qml.qnode(dev, interface="torch")
def quantum_net(inputs, new_weights):
    
    wirelist = [i for i in range(n_qubits)]

    #Encode the data with Angle Embedding
    qml.templates.AngleEmbedding(inputs, wires=wirelist, rotation='X')
    
    #apply frozen layers
    frozen_layers(layer_gates, layer_weights)

    for i in range(n_layers_to_add):
        apply_layer(new_gates[i], new_weights[i])
        
    # Expectation value of the last qubit
    return qml.expval(qml.PauliZ(n_qubits-1))

In [ ]:
layer_gates = []
layer_weights = []
sigmoid = nn.Sigmoid()
for step in range(n_layer_steps):
    
    print(f"Phase I step: {step+1}")
    new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
    
    # Define shape of the weights
    weight_shapes = {"new_weights": (n_layers_to_add, n_qubits)}
    
    # Quantum net as a TorchLayer
    qlayer = qml.qnn.TorchLayer(quantum_net, weight_shapes, init_method = nn.init.zeros_)
    
    # Create Sequential Model
    model = torch.nn.Sequential(qlayer)
    
    # Optimization and loss
    opt = optim.Adam(model.parameters(), lr=0.01)

    loss = nn.HingeEmbeddingLoss()
    
    batches = NUM_EXAMPLES // batch_size
    for epoch in range(epochs):
        running_loss = 0
        for x, y in train_loader:
            opt.zero_grad()
            y = y.to(torch.float32)
            loss_evaluated = loss(model(x), y)
            loss_evaluated.backward()
            running_loss += loss_evaluated

            opt.step()
        avg_loss = running_loss / batches
        print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
    
    for param in model.parameters():
        new_weights = param.data
    new_weights = new_weights.tolist()
    print(f"Trained parameters: {total_elements(new_weights)}")

    layer_gates += new_gates
    layer_weights += new_weights
    print(f"Layer weights: {total_elements(layer_weights)}")
    print("")

Phase I step: 1
Average loss over epoch 1: 0.5145
Average loss over epoch 2: 0.5144
Average loss over epoch 3: 0.5143
Average loss over epoch 4: 0.5142
Average loss over epoch 5: 0.5141
Average loss over epoch 6: 0.5139
Average loss over epoch 7: 0.5138
Average loss over epoch 8: 0.5137
Average loss over epoch 9: 0.5135
Average loss over epoch 10: 0.5134
Average loss over epoch 11: 0.5132
Average loss over epoch 12: 0.5131
Average loss over epoch 13: 0.5129


## Phase II

In [ ]:
partition_percentage = 0.5
partition_size = int(n_layer_steps*n_layers_to_add*partition_percentage)
n_partition_weights = partition_size*n_qubits
n_sweeps = 2

In [ ]:
@qml.qnode(dev, interface="torch")
def train_partition(inputs, partition_weights):
    
    wirelist = [i for i in range(n_qubits)]

    #Encode the data with Angle Embedding
    qml.templates.AngleEmbedding(inputs, wires=wirelist, rotation='X')
    
    if partition == 1:
        # Apply trainable partition first
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], partition_weights[i])
        
        #Apply non-trainable partition
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], layer_weights[partition_size:][i])
    
    elif partition == 2:
        # Apply non-trainable partition first
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], layer_weights[:partition_size][i])
        
        # Apply trainable partition
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], partition_weights[i])
    
    # Expectation value of the last qubit
    return qml.expval(qml.PauliZ(n_qubits-1))

In [ ]:
for sweep in range(n_sweeps):
    
    partition = 1
    print(f"Sweep: {sweep+1}, partition: {partition}")
    trainable_weights = layer_weights[:partition_size]
    
    # Define shape of the weights
    weight_shapes = {"partition_weights": (len(trainable_weights), n_qubits)}
    
    # Quantum net as a TorchLayer
    qlayer = qml.qnn.TorchLayer(train_partition, weight_shapes, init_method = nn.init.zeros_)
    
    init_weights = nn.Parameter(torch.tensor(trainable_weights))
    
    # Create Sequential Model
    model = torch.nn.Sequential(qlayer)
    
    # Edit model parameters to be init_weights
    old_params = {}
    for name, params in model.named_parameters():
        old_params[name] = params.clone()
    
    old_params["0.partition_weights"] = init_weights
    
    for name, params in model.named_parameters():
        params.data.copy_(old_params[name])

    # Optimization and loss
    opt = optim.Adam(model.parameters(), lr=0.01)

    loss = nn.HingeEmbeddingLoss()
    
    batches = NUM_EXAMPLES // batch_size
    for epoch in range(epochs):
        running_loss = 0
        for x, y in train_loader:
            opt.zero_grad()
            y = y.to(torch.float32)
            loss_evaluated = loss(model(x), y)
            loss_evaluated.backward()
            running_loss += loss_evaluated

            opt.step()
        avg_loss = running_loss / batches
        print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
    
    for param in model.parameters():
        trained_weights = param.data
    trained_weights = trained_weights.tolist()
    print(f"Trained parameters: {total_elements(trained_weights)}")

    layer_weights[:partition_size] = trained_weights
    
    partition = 2
    print(f"Sweep: {sweep+1}, partition: {partition}")
    trainable_weights = layer_weights[partition_size:]
    
    # Define shape of the weights
    weight_shapes = {"partition_weights": (len(trainable_weights), n_qubits)}

    # Quantum net as a TorchLayer
    qlayer = qml.qnn.TorchLayer(train_partition, weight_shapes, init_method = nn.init.zeros_)
    
    init_weights = nn.Parameter(torch.tensor(trainable_weights))

    # Create Sequential Model
    model = torch.nn.Sequential(qlayer)
    
    # Edit model parameters to be init_weights
    old_params = {}
    for name, params in model.named_parameters():
        old_params[name] = params.clone()
    
    old_params["0.partition_weights"] = init_weights
    
    for name, params in model.named_parameters():
        params.data.copy_(old_params[name])
    
    # Optimization and loss
    opt = optim.Adam(model.parameters(), lr=0.01)

    loss = nn.HingeEmbeddingLoss()
    
    batches = NUM_EXAMPLES // batch_size
    for epoch in range(epochs):
        running_loss = 0
        for x, y in train_loader:
            opt.zero_grad()
            y = y.to(torch.float32)
            loss_evaluated = loss(model(x), y)
            loss_evaluated.backward()
            running_loss += loss_evaluated

            opt.step()
        avg_loss = running_loss / batches
        print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
    
    for param in model.parameters():
        trained_weights = param.data
    trained_weights = trained_weights.tolist()
    print(f"Trained parameters: {total_elements(trained_weights)}")

    layer_weights[partition_size:] = trained_weights

In [ ]:
for x, y in train_loader:
    print(model(x))
    print(y)